In [89]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'auctions.game' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'auctions.experiment' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'auctions.equilibriumsolverrun' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.regis

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path
import scipy.stats
import time

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM
from open_spiel.python.examples.ubc_utils import *
import open_spiel.python.examples.ubc_dispatch as dispatch

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from open_spiel.python.examples.ubc_cma import *

output_notebook()
from open_spiel.python.games.clock_auction_base import InformationPolicy
from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator

Loading BokehJS ...

In [91]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]
PREFIX = 'jun24'
sats_games = sorted([f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*') if 'base' in x])


In [80]:
df, combo_to_score, scorer = efficient_allocation(game, factor_in_opening_prices=True, verbose=False)

In [92]:
records = []
for game_name in sats_games:
    game = pyspiel.load_game('python_clock_auction', dict(filename=game_name+'.json'))
    record = dict(name=game_name, max_straightforward=max_straightfoward_rounds(game))
    records.append(record)
df = pd.DataFrame.from_records(records)

In [93]:
df

,name,max_straightforward
0,jun24/jun24_0_base,5
1,jun24/jun24_1_base,6
2,jun24/jun24_2_base,4
3,jun24/jun24_3_base,4
4,jun24/jun24_4_base,6


In [94]:
records = []
for game_name in sats_games:
    game = pyspiel.load_game('python_clock_auction', dict(filename=game_name+'.json'))
    record = dict(name=game_name, max_straightforward=max_straightfoward_rounds(game, alternating=True, num_samples=100))
    records.append(record)
df = pd.DataFrame.from_records(records)
df

,name,max_straightforward
0,jun24/jun24_0_base,7
1,jun24/jun24_1_base,7
2,jun24/jun24_2_base,7
3,jun24/jun24_3_base,5
4,jun24/jun24_4_base,6
